In [1]:
#@title Global Settings
from tensorflow import keras
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.datasets
from tensorflow.keras import regularizers
dataset=tensorflow.keras.datasets.cifar10 #@param ["tensorflow.keras.datasets.cifar10", "tensorflow.keras.datasets.cifar100", "tensorflow.keras.datasets.mnist", "tensorflow.keras.datasets.fashion_mnist"] {type:"raw"} 
batch_size=64 # @param [16, 32, 64, 128, 256, 512] {type:"raw"} 
channels_per_group=32 # @param [2, 4, 8, 12, 16, 32] {type:"raw"}
epochs=50 # @param [2, 25, 50, 75, 100, 100, 200, 400] {type:"raw"}
verbose=True #@param {type:"boolean"}
bipolar_input=True #@param {type:"boolean"}
seed=7

if dataset is keras.datasets.cifar10 or dataset is keras.datasets.cifar100:
  global_input_shape = (32, 32, 3)
else:
  global_input_shape = (28, 28, 1)

if (dataset==keras.datasets.cifar100):
  num_classes = 100
else:
  num_classes = 10

In [2]:
import os
if not os.path.isdir('k'):
    !git clone -b development16 https://github.com/joaopauloschuler/k-neural-api.git k
else:
    !cd k && git pull
!cd k && pip install .

Already up to date.
Processing c:\users\stan_\lidc-idri-preproc\diploma_work\efficientnet\k
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.0.1-cp39-cp39-win_amd64.whl (10.7 MB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached scikit_learn-1.2.2-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Created wheel for cai: filename=cai-0.1.7-py3-none-any.whl size=61790 sha256=4ef7e7d944dd77127e8656d994f966faa75f039829e809de4043141aaaeae1bb
  Stored in directory: C:\Users\stan_\AppData\Local\Temp\pip-ephem-wheel-cache-fx7d03vu\wheels\7d\bf\8a\a9657c0c9e5ef314407831cf13e1c2662ff36e8b5430a7e706
Successfully built cai


In [4]:
import cai.layers
import cai.datasets
import cai.efficientnet
import cai.util
import gc
import multiprocessing
import random
import tensorflow as tf
import numpy as np
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
print("CPU cores:", multiprocessing.cpu_count())
import psutil
print('RAM:', (psutil.virtual_memory().total / 1e9),'GB')
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pylab as plt
from sklearn.metrics import classification_report

Tensorflow version: 2.12.0
Keras version: 2.12.0
CPU cores: 8
RAM: 16.859602944 GB
[]


Matplotlib is building the font cache; this may take a moment.


In [4]:
#%conda install -c conda-forge opencv

In [5]:
if channels_per_group==2:
   kTypes = [cai.layers.D6v3_2ch()]
elif channels_per_group==4:
   kTypes = [cai.layers.D6v3_4ch()]
elif channels_per_group==8:
   kTypes = [cai.layers.D6v3_8ch()]
elif channels_per_group==12:
   kTypes = [cai.layers.D6v3_12ch()]
elif channels_per_group==16:
   kTypes = [cai.layers.D6v3_16ch()]
else:
   kTypes = [cai.layers.D6v3_32ch()]

In [6]:
!nvidia-smi

"nvidia-smi" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [7]:
base_model_name='kEffNetV2Example'
x_train, y_train, x_val, y_val, x_test, y_test = cai.datasets.load_dataset_with_validation(dataset,
  lab=False, verbose=verbose, bipolar=bipolar_input,
  base_model_name=base_model_name,
  validation_size=0.1, validation_flip_horizontal=True,
  validation_flip_vertical=False)
print("Train shapes:", x_train.shape, y_train.shape)
print("Validation shapes:", x_val.shape, y_val.shape)
print("Test shapes:", x_test.shape, y_test.shape)

train shape (50000, 32, 32, 3)
test shape (10000, 32, 32, 3)
Original channel  0  min: 0.0  max: 255.0
Original channel  1  min: 0.0  max: 255.0
Original channel  2  min: 0.0  max: 255.0
Loading RGB.
Channel  0  min: -2.0  max: 1.984375
Channel  1  min: -2.0  max: 1.984375
Channel  2  min: -2.0  max: 1.984375
Train shapes: (45000, 32, 32, 3) (45000, 10)
Validation shapes: (10000, 32, 32, 3) (10000, 10)
Test shapes: (10000, 32, 32, 3) (10000, 10)


In [9]:
train_datagen = cai.util.create_image_generator(rotation_range=20, 
  width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.0)
valid_datagen = cai.util.create_image_generator_no_augmentation()
test_datagen = cai.util.create_image_generator_no_augmentation()
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [10]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False, kTypes=[]):
  monitor='val_loss'
  if (show_model):
    input_shape = global_input_shape
  else:
    input_shape = (None, None, global_input_shape[2])
  for kType in kTypes:
      basefilename = 'kEffNetV2-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      model = cai.efficientnet.kEfficientNetB0(
        include_top=True,
        skip_stride_cnt=3,
        input_shape=input_shape,
        classes=num_classes,
        kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): 
        model.summary(line_length=180)

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            save_freq='epoch')

      if (run_fit): 
            train_flow = train_datagen.flow(
                x_train, y_train,
                batch_size=batch_size,
                shuffle=True,
                seed=seed
            )
            validation_flow = valid_datagen.flow(
                x_val, y_val,
                batch_size=batch_size,
                shuffle=False,
                seed=seed
            )
            history = model.fit(
              x = train_flow,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=validation_flow,
              callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
              workers=cpus_num,
              max_queue_size=128
            )
            plt.figure()
            plt.ylabel("Accuracy (training and validation)")
            plt.xlabel("Epochs")
            plt.ylim([0,1])
            plt.plot(history.history["accuracy"])
            plt.plot(history.history["val_accuracy"])
      if (test_results):
        test_flow = test_datagen.flow(
            x_test, y_test,
            batch_size=batch_size,
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        cai.datasets.test_flips_on_saved_model(x_test, y_test, best_result_file_name, has_flip_x=True, has_flip_y=True, has_bw=False, center_crop=0.15)
      print('Finished: '+basefilename)

In [11]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False, kTypes=kTypes)

Running: kEffNetV2-33
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 32), dtype=tf.float32, name=None), name='k_block1a__0se_reshape/Reshape:0', description="created by layer 'k_block1a__0se_reshape'")
prev_layer_channel_count  32
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 8), dtype=tf.float32, name=None), name='k_block1a__0se_reduce_um/mul:0', description="created by layer 'k_block1a__0se_reduce_um'")
prev_layer_channel_count  8
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 31, 31, 32), dtype=tf.float32, name=None), name='k_block1a__0se_excite/mul:0', description="created by layer 'k_block1a__0se_excite'")
prev_layer_channel_count  32
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 31, 31, 16), dtype=tf.float32, name=None), name='k_block1a__0project_conv_um_bn/FusedBatchNormV3:0', description="created by layer 'k_block1a__0project_conv_um_bn'")
prev_layer_channel_count  16
last_tensor   KerasTensor(type_spec=TensorSpe

In [12]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=False, kTypes=kTypes)

Running: kEffNetV2-33
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 32), dtype=tf.float32, name=None), name='k_block1a__0se_reshape/Reshape:0', description="created by layer 'k_block1a__0se_reshape'")
prev_layer_channel_count  32
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 8), dtype=tf.float32, name=None), name='k_block1a__0se_reduce_um/mul:0', description="created by layer 'k_block1a__0se_reduce_um'")
prev_layer_channel_count  8
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 32), dtype=tf.float32, name=None), name='k_block1a__0se_excite/mul:0', description="created by layer 'k_block1a__0se_excite'")
prev_layer_channel_count  32
last_tensor   KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 16), dtype=tf.float32, name=None), name='k_block1a__0project_conv_um_bn/FusedBatchNormV3:0', description="created by layer 'k_block1a__0project_conv_um_bn'")
prev_layer_channel_count  16
last_tensor   KerasTensor(type_spec=T

KeyboardInterrupt: 

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True, kTypes=kTypes)

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=True, kTypes=kTypes)